Film Önericisi sistemimiz için ilk olarak pandas ve numpy kütüphanelerini import ediyoruz. Ayrıca daha sonra kullanmak üzere sklearn'den countVectorizer ve cosine_similarity'i de import ediyoruz. 

In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

movies_url = "https://raw.githubusercontent.com/kaveai/arge-odakli-yapay-zeka-egitimi2-sorulari/main/datasets/movies.csv"
ratings_url = "https://raw.githubusercontent.com/kaveai/arge-odakli-yapay-zeka-egitimi2-sorulari/main/datasets/ratings.csv"


movies ve ratings değişkenleriyle tanımladığımız DataFrame'leri merge ile birleştiriyoruz: 

In [2]:
movies = pd.read_csv(movies_url)
ratings = pd.read_csv(ratings_url) 
df = pd.merge(ratings, movies,how="left", on="movieId") 
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Ben genres sütunundaki kelimeleri ayrı ayrı görebilmek adına, arada pipe (|) olmayacak şekilde düzenlemek için bir fonksiyon yazdım (Ayrıca cosine similarity kullanacağımız zaman bu kelimelerin ayrı ayrı, düzgün şekilde dataframe'de bulunuyor olması gerek)



In [3]:
genresSeperated = []
for i in  df["genres"]:
   i = i.replace("|", " ")
   genresSeperated.append(i)
   
genresSeperated = tuple(genresSeperated)

df["genresSeperated"]=genresSeperated
df.drop("genres", axis=1, inplace=True)

df.head()
    

,userId,movieId,rating,timestamp,title,genresSeperated
0,1,1,4.0,964982703,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy Romance
2,1,6,4.0,964982224,Heat (1995),Action Crime Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime Mystery Thriller


Daha sonra filmler arasında çekim yılı açısından da bir benzerlik kurabilmek adına film isimlerinden yıllarını ayırarak ayrı bir sütun olarak df'e ekliyorum.

In [4]:
year = []
for i in df["title"]:
    i = i[-5:-1]
    year.append(i)
year = tuple(year)

df["year"]=year
df.head()

,userId,movieId,rating,timestamp,title,genresSeperated,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy Romance,1995
2,1,6,4.0,964982224,Heat (1995),Action Crime Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery Thriller,1995
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime Mystery Thriller,1995


Yıl, türler ve puanlama bizim kullanacağımız esas parametler. Bunun için fillna ile eğer NaN değer varsa dolduruyoruz.

In [5]:
features = ["year", "genresSeperated", "rating"]
for feature in features: 
    df[feature] = df[feature].fillna("")

Daha sonra bu özelliklerin hepsini ayrı bir sütunda bir araya getiriyoruz.

In [6]:
def combined_features(row):
    return str(row["year"]) +" "+row["genresSeperated"]+ " "+ str(row["rating"])

df["combined_features"] = df.apply(combined_features, axis=1)
df.head()

,userId,movieId,rating,timestamp,title,genresSeperated,year,combined_features
0,1,1,4.0,964982703,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,1995 Adventure Animation Children Comedy Fanta...
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy Romance,1995,1995 Comedy Romance 4.0
2,1,6,4.0,964982224,Heat (1995),Action Crime Thriller,1995,1995 Action Crime Thriller 4.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery Thriller,1995,1995 Mystery Thriller 5.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime Mystery Thriller,1995,1995 Crime Mystery Thriller 5.0


CountVectorizer ile combined_features sütunumuzdaki eşleşmelerin her biri için bir puan veriliyor. countMatrix ie de bu değerleri bir matris içerisinde gösteriyoruz. 

In [55]:
cv = CountVectorizer()
countMatrix = cv.fit_transform(df["combined_features"])
print("Count Matrix: ", countMatrix.toarray())

Count Matrix:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


Benzerlik bulmak için asıl kilit nokta kosinüs benzerliğini kullanmak. Kosinüs benzerliği 1'e ne kadar yakınsa bu değerler her bir satırdaki film ile ne kadar benziyor demek. Fakat elde ettiğimiz bu tablo çok, çok büyük bir tablo. O sebeple benim programım -bilgisayardan kaynaklı olduğunu varsayıyorum- çok fazla RAM kullanması sebebiyle kernel çökmesiyle karşılaştı ve buna ancak, bu matrisin yalnızca ilk 20.000 satırını kullanarak çözüm bulabildim. 
Yalnızca sınırla sayıda bir veriyi kullacağımız için elde edeceğimiz "Benzer Filmler" listesi de bir hayli kısa olacak.

In [8]:

  cosineSim = cosine_similarity(countMatrix[0:20000])
  cosineSim
  


array([[1.        , 0.47140452, 0.20412415, ..., 0.23570226, 0.        ,
        0.54772256],
       [0.47140452, 1.        , 0.28867513, ..., 0.33333333, 0.33333333,
        0.25819889],
       [0.20412415, 0.28867513, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.23570226, 0.33333333, 0.        , ..., 1.        , 0.66666667,
        0.25819889],
       [0.        , 0.33333333, 0.        , ..., 0.66666667, 1.        ,
        0.        ],
       [0.54772256, 0.25819889, 0.        , ..., 0.25819889, 0.        ,
        1.        ]])

Filmin adını girdiğimizde lowercase, uppercase olmasına bakılmaksızın ve filmin verisetinde yer aldığı şekliyle tarihiyle beraber yazılıp yazılmaksızın, filmin veri setindeki orijinal ismini verecek bir fonksiyon yazdım.

In [9]:
def giveFullName(name): 
    name = name.lower()
    for i in enumerate(df['title']):
        fullMovieTitle = i[1]
        shortMovieTitle = fullMovieTitle[:-7]
        shortMovieTitle = shortMovieTitle.lower()
        
        if(name==shortMovieTitle): 
            name = fullMovieTitle
    return name

def getTitle(movieId):
    return df[df.movieId == movieId]["title"].values[0]

def getIndex(title):
    title = giveFullName(title)
    return df[df.title == title]["movieId"].values[0]

getIndex("iron man")


59315

Benzer filmlerin en yüksekten en düşük puana göre listesini verecek olan bu fonksiyonda, countMatrix'in yalnızca belli bir kısmını kullandığımız için hata almamak adına try-except bloku yazdım. Gördüğünüz üzere yine aynı sebepten 15 adet değil genelde daha az sonuç gösteriyor: 

In [56]:
def filmoner(movieTitle): 
  movieId = getIndex(movieTitle)
  similarMovies = list(enumerate(cosineSim[movieId]))
  
  SimilarMoviesSorted = sorted(similarMovies, key=lambda x:x[1], reverse=True)

  i=0
  try: 
    for movie in SimilarMoviesSorted:
      id = movie[0]
      print(getTitle(id))
      i=i+1
      if i>15:
        break
  except: 
    pass

In [57]:
filmoner("jumanji")

Jumanji (1995)
Chasers (1994)
Thin Line Between Love and Hate, A (1996)
Family Thing, A (1996)
